# Applicazione Python "Premi Nobel"

Attività scelta: 3. MongoDB Usecase e applicazione Python
Progetto realizzato da Elena Curti (matricola: 146539 <mark>FORSE DA CAMBIARE?</mark>) con Python 3.11.0

Ho usato i seguenti file JSON pubblici: <br>
    <ul> 
      <li> nobelPrizes.json: https://masterdataapi.nobelprize.org/2.1/nobelPrizes?offset=0&limit=664 </li>
      <li> laureates.json: https://masterdataapi.nobelprize.org/2.1/laureates?offset=0&limit=981 </li>
    </ul>
Essi contengono informazioni sui premi Nobel ed i relativi vincitori.

## Documentazione 
Documentazione dei dataset: https://app.swaggerhub.com/apis/NobelMedia/NobelMasterData/2.1


## File nobelPrizes.json
Ho modificato il file nobelPrizes.json in modo da realizzare un file contenente l'elenco dei premi Nobel, con la seguente struttura d'esempio:
{
    "awardYear": "2021",
    "category": "Economic Sciences",
    "categoryFullName": "The Sveriges Riksbank Prize in Economic Sciences in Memory of Alfred Nobel",
    "dateAwarded": "2021-10-11",
    "topMotivation": "for contributions ...",
    "prizeAmount": 10000000,
    "prizeAmountAdjusted": 10000000,
    "laureates": [
        {
            "id": "1007",
            "knownName": "David Card",
            "fullName": "David Card",
            "portion": "1/2",
            "sortOrder": "1",
            "motivation": "for his empirical contributions to labour economics"
        },
        { ... }
    ]
}


<br>



In [1]:
print("ciao")

ciao
